In [1]:
!pip install ../input/packages/pretrainedmodels-0.7.4-py3-none-any.whl
!pip install ../input/segmentationmodelspytorch/segmentation_models/timm-0.1.20-py3-none-any.whl
!pip install ../input/packages/efficientnet_pytorch-0.6.3-py2.py3-none-any.whl
!pip install ../input/segmentationmodelspytorch/segmentation_models/segmentation_models_pytorch-0.1.2-py3-none-any.whl

Processing /kaggle/input/packages/pretrainedmodels-0.7.4-py3-none-any.whl
Processing /kaggle/input/segmentationmodelspytorch/segmentation_models/timm-0.1.20-py3-none-any.whl
Processing /kaggle/input/packages/efficientnet_pytorch-0.6.3-py2.py3-none-any.whl
Processing /kaggle/input/segmentationmodelspytorch/segmentation_models/segmentation_models_pytorch-0.1.2-py3-none-any.whl


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hubmap-kidney-segmentation/sample_submission.csv
/kaggle/input/hubmap-kidney-segmentation/HuBMAP-20-dataset_information.csv
/kaggle/input/hubmap-kidney-segmentation/train.csv
/kaggle/input/hubmap-kidney-segmentation/test/aa05346ff.tiff
/kaggle/input/hubmap-kidney-segmentation/test/sample_submission.csv
/kaggle/input/hubmap-kidney-segmentation/test/3589adb90-anatomical-structure.json
/kaggle/input/hubmap-kidney-segmentation/test/2ec3f1bb9.tiff
/kaggle/input/hubmap-kidney-segmentation/test/3589adb90.tiff
/kaggle/input/hubmap-kidney-segmentation/test/d488c759a-anatomical-structure.json
/kaggle/input/hubmap-kidney-segmentation/test/57512b7f1-anatomical-structure.json
/kaggle/input/hubmap-kidney-segmentation/test/d488c759a.tiff
/kaggle/input/hubmap-kidney-segmentation/test/57512b7f1.tiff
/kaggle/input/hubmap-kidney-segmentation/test/aa05346ff-anatomical-structure.json
/kaggle/input/hubmap-kidney-segmentation/test/2ec3f1bb9-anatomical-structure.json
/kaggle/input/hubmap-kidney-

In [3]:
import random
import time
import warnings
warnings.simplefilter("ignore")

import os
from albumentations import *
from albumentations.pytorch import ToTensor
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import segmentation_models_pytorch as smp
from sklearn.model_selection import KFold
import tifffile as tiff
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, sampler
from tqdm import tqdm_notebook as tqdm
import gc


In [4]:
def UnetDenseNet():
    return smp.Unet(
    encoder_name='densenet201',
    encoder_weights=None,
    in_channels=3,
    classes=1)

In [5]:
model = UnetDenseNet().cuda()

In [6]:
model = torch.load('../input/base-network/best_unet_model.pth')

In [7]:
gc.collect()

22

In [8]:
#test open file

In [9]:
df = pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')
df.head()

,id,predicted
0,2ec3f1bb9,NaN
1,3589adb90,NaN
2,d488c759a,NaN
3,aa05346ff,NaN
4,57512b7f1,NaN


In [10]:
fold = 0
nfolds = 5
reduce = 4
sz = 256

BATCH_SIZE = 16
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
NUM_WORKERS = 4
SEED = 2020
TH = 0.50  #threshold for positive predictions

DATA = '../input/hubmap-kidney-segmentation/test/'
LABELS = '../input/hubmap-kidney-segmentation/train.csv'

In [11]:
# https://www.kaggle.com/iafoss/256x256-images
mean = np.array([0.63701495, 0.4709702,  0.6817423] )
std = np.array([0.15978882, 0.2245109, 0.14173926])

def img2tensor(img,dtype:np.dtype=np.float32):
    if img.ndim==2 : img = np.expand_dims(img,2)
    img = np.transpose(img,(2,0,1))
    return torch.from_numpy(img.astype(dtype, copy=False))

class HuBMAPTestDataset(Dataset):
    def __init__(self, idx, sz=sz, reduce=reduce):
        self.data = rasterio.open(os.path.join(DATA,idx+'.tiff'), transform = identity,
                                 num_threads='all_cpus')
        # some images have issues with their format 
        # and must be saved correctly before reading with rasterio
        if self.data.count != 3:
            subdatasets = self.data.subdatasets
            self.layers = []
            if len(subdatasets) > 0:
                for i, subdataset in enumerate(subdatasets, 0):
                    self.layers.append(rasterio.open(subdataset))
        self.shape = self.data.shape
        self.reduce = reduce
        self.sz = reduce*sz
        self.pad0 = (self.sz - self.shape[0]%self.sz)%self.sz
        self.pad1 = (self.sz - self.shape[1]%self.sz)%self.sz
        self.n0max = (self.shape[0] + self.pad0)//self.sz
        self.n1max = (self.shape[1] + self.pad1)//self.sz
        
    def __len__(self):
        return self.n0max*self.n1max
    
    def __getitem__(self, idx):
        # the code below may be a little bit difficult to understand,
        # but the thing it does is mapping the original image to
        # tiles created with adding padding, as done in
        # https://www.kaggle.com/iafoss/256x256-images ,
        # and then the tiles are loaded with rasterio
        # n0,n1 - are the x and y index of the tile (idx = n0*self.n1max + n1)
        n0,n1 = idx//self.n1max, idx%self.n1max
        # x0,y0 - are the coordinates of the lower left corner of the tile in the image
        # negative numbers correspond to padding (which must not be loaded)
        x0,y0 = -self.pad0//2 + n0*self.sz, -self.pad1//2 + n1*self.sz
        # make sure that the region to read is within the image
        p00,p01 = max(0,x0), min(x0+self.sz,self.shape[0])
        p10,p11 = max(0,y0), min(y0+self.sz,self.shape[1])
        img = np.zeros((self.sz,self.sz,3),np.uint8)
        # mapping the loade region to the tile
        if self.data.count == 3:
            img[(p00-x0):(p01-x0),(p10-y0):(p11-y0)] = np.moveaxis(self.data.read([1,2,3],
                window=Window.from_slices((p00,p01),(p10,p11))), 0, -1)
        else:
            for i,layer in enumerate(self.layers):
                img[(p00-x0):(p01-x0),(p10-y0):(p11-y0),i] =\
                  layer.read(1,window=Window.from_slices((p00,p01),(p10,p11)))
        
        if self.reduce != 1:
            img = cv2.resize(img,(self.sz//reduce,self.sz//reduce),
                             interpolation = cv2.INTER_AREA)
        #check for empty imges
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h,s,v = cv2.split(hsv)
        if (s>s_th).sum() <= p_th or img.sum() <= p_th:
            #images with -1 will be skipped
            return img2tensor((img/255.0 - mean)/std), -1
        else: return img2tensor((img/255.0 - mean)/std), idx

In [12]:
#https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
#with bug fix
def rle_encode_less_memory(img):
    #watch out for the bug
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

In [13]:
#iterator like wrapper that returns predicted masks
class Model_pred:
    def __init__(self, models, dl, tta:bool=True, half:bool=False):
        self.models = models
        self.dl = dl
        self.tta = tta
        self.half = half
        
    def __iter__(self):
        count=0
        with torch.no_grad():
            for x,y in iter(self.dl):
                if ((y>=0).sum() > 0): #exclude empty images
                    x = x[y>=0].to(DEVICE)
                    y = y[y>=0]
                    if self.half: x = x.half()
                    py = None
                    for model in self.models:
                        p = model(x)
                        p = torch.sigmoid(p).detach()
                        if py is None: py = p
                        else: py += p
                    if self.tta:
                        #x,y,xy flips as TTA
                        flips = [[-1],[-2],[-2,-1]]
                        for f in flips:
                            xf = torch.flip(x,f)
                            for model in self.models:
                                p = model(xf)
                                p = torch.flip(p,f)
                                py += torch.sigmoid(p).detach()
                        py /= (1+len(flips))        
                    py /= len(self.models)

                    py = F.upsample(py, scale_factor=reduce, mode="bilinear")
                    py = py.permute(0,2,3,1).float().cpu()
                    
                    batch_size = len(py)
                    for i in range(batch_size):
                        yield py[i],y[i]
                        count += 1
                    
    def __len__(self):
        return len(self.dl.dataset)

In [14]:
models = [model]

In [15]:
import rasterio
from rasterio.windows import Window

In [16]:
gc.collect()

26

In [17]:
s_th = 40  #saturation blancking threshold
p_th = 1000*(sz//256)**2 #threshold for the minimum number of pixels
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)

names,preds = [],[]
print(df)
for idx,row in tqdm(df.iterrows(),total=len(df)):
    idx = row['id']
    ds = HuBMAPTestDataset(idx)
    #rasterio cannot be used with multiple workers
    dl = DataLoader(ds,BATCH_SIZE,num_workers=0,shuffle=False,pin_memory=True)
    mp = Model_pred(models,dl)
    #generate masks
    mask = torch.zeros(len(ds),ds.sz,ds.sz,dtype=torch.int8)
    for p,i in iter(mp): mask[i.item()] = p.squeeze(-1) > TH
    
    #reshape tiled masks into a single mask and crop padding
    mask = mask.view(ds.n0max,ds.n1max,ds.sz,ds.sz).\
        permute(0,2,1,3).reshape(ds.n0max*ds.sz,ds.n1max*ds.sz)
    mask = mask[ds.pad0//2:-(ds.pad0-ds.pad0//2) if ds.pad0 > 0 else ds.n0max*ds.sz,
        ds.pad1//2:-(ds.pad1-ds.pad1//2) if ds.pad1 > 0 else ds.n1max*ds.sz]
    
    #convert to rle
    #https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
    rle = rle_encode_less_memory(mask.numpy())
    names.append(idx)
    preds.append(rle)
    del mask, ds, dl
    gc.collect()

          id  predicted
0  2ec3f1bb9        NaN
1  3589adb90        NaN
2  d488c759a        NaN
3  aa05346ff        NaN
4  57512b7f1        NaN


  0%|          | 0/5 [00:00<?, ?it/s]

In [18]:
df = pd.DataFrame({'id':names,'predicted':preds})
df.to_csv('submission.csv',index=False)

In [19]:
df.head()

,id,predicted
0,2ec3f1bb9,60642347 2 60666327 27 60690310 46 60714292 63...
1,3589adb90,68570660 14 68600088 23 68629517 31 68658947 3...
2,d488c759a,271721609 9 271768267 12 271814926 14 27186158...
3,aa05346ff,53041035 13 53071746 27 53102462 35 53133178 4...
4,57512b7f1,279956105 3 279989338 15 280022576 20 28005581...
